In [1]:
%pip install azureml-widgets
%pip install azureml-sdk[notebooks]

StatementMeta(c38271bb-6ebc-4469-90f0-bf01f6a3d71c, 2, 11, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 104.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 114.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 KB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.4/617.4 KB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 KB 45.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.4
    Not uninstalling urllib3 at /home/trusted-service-user/cluster-env/env/lib/python3.10/site-packages, outside environment /nfs4/pyenv-03400fbd-2c32-464e-a2a6-480866e7663a
    Can't uninstall 'urllib3'. No files were found to uninstall.
  Attempting uninstall: isodate
    Found existing installation: isodate 0.6.0
    Not uninstalling isodate at /home/trusted-service-user/cluster-env/env/lib/python3.10/site-packages, ou

In [2]:
from azureml.core import Workspace, Experiment

# ws = Workspace.from_config()
subscription_name = "quick-starts-ws-266991"
subscription_id='61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30'

ws = Workspace.get(name = subscription_name,
subscription_id=subscription_id,
resource_group="aml-quickstarts-266991"
)


exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')


run = exp.start_logging()

StatementMeta(c38271bb-6ebc-4469-90f0-bf01f6a3d71c, 2, 17, Finished, Available, Finished)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code CRK696NPQ to authenticate.
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-266991
Azure region: westeurope
Subscription id: 61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30
Resource group: aml-quickstarts-266991


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = subscription_name

vm_size = "Standard_D2_V2"
compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=4)
#Create the compute cluster
compute_target = ComputeTarget.create(ws,cluster_name, compute_config)
#wait for the compute ready
compute_target.wait_for_completion(show_output=True)

StatementMeta(c38271bb-6ebc-4469-90f0-bf01f6a3d71c, 2, 18, Finished, Available, Finished)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [15]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {"C": uniform(0.05, 0.1),
    "max_iter" : choice(16, 32, 64, 128)}
)

# Specify a Policy
policy = BanditPolicy(slack_factor=0.2, evaluation_interval=1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', 
file_path='Users/odl_user_266991/conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory=".",
    script="Users/odl_user_266991/train.py",
    environment=sklearn_env,
    compute_target=compute_target
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="AUC",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=10,
    max_concurrent_runs=4
)

StatementMeta(c38271bb-6ebc-4469-90f0-bf01f6a3d71c, 2, 30, Finished, Available, Finished)

In [16]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

experiment = Experiment(workspace=ws, name="my-hyperdive-experiment")
experiment_run = experiment.submit(hyperdrive_config)
RunDetails(experiment_run).show()

StatementMeta(c38271bb-6ebc-4469-90f0-bf01f6a3d71c, 2, 31, Finished, Available, Finished)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [11]:
all_runs = experiment.get_runs()
for run in all_runs:
    print(run.id, run.status)

StatementMeta(c38271bb-6ebc-4469-90f0-bf01f6a3d71c, 2, 26, Finished, Available, Finished)

HD_76f2ac5e-8e79-4a19-a6c8-1ddbadeef101 Completed
HD_074137b6-c9ca-4b71-beb3-33f5d0ff8e6f Failed
HD_d9f3f607-8481-4e83-97c3-1cdbe834942f Failed
HD_e10f42c4-45b0-4890-878d-00d1f6929031 Failed
HD_660c1059-a70a-469e-9c5f-2d933d0bea42 Failed
HD_2ad6e0be-2990-4f2a-aa94-3d459da231ce Failed
HD_949672cd-9f23-4671-979a-f2304dd3c493 Failed
HD_be537ba0-9fdd-476a-a082-7e2895a7e91d Failed
HD_dcdf175f-4775-44b5-bcf0-1dd737f554b5 Failed
HD_aec21f4d-3011-4dfc-b739-bafdc0190e09 Failed


In [12]:
import joblib
# Get your best run and save the model from that run.

best_run = experiment_run.get_best_run_by_primary_metric()
# model= best_run.register_model(model_name="best_model",
# model_path="outputs/best_model.pkl")

StatementMeta(c38271bb-6ebc-4469-90f0-bf01f6a3d71c, 2, 27, Finished, Available, Finished)

In [14]:
print(best_run.get_metrics())

StatementMeta(c38271bb-6ebc-4469-90f0-bf01f6a3d71c, 2, 29, Finished, Available, Finished)

{'Regularization Strength:': 0.055078151873633, 'Max iterations:': 128, 'Accuracy': 0.907587253414264}


In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###